In [2]:
from urllib.parse import quote_plus
import mysql.connector
from mysql.connector import Error
import pandas as pd
from sqlalchemy import create_engine,text

In [3]:
def connection(hostname, username, password, dbname):
    # Initialize cinnection to None
    con = None

    # Encode the password
    password = quote_plus(password)

    # Create connection while checking for any errors
    try:
        con = mysql.connector.connect(host=hostname, user=username, passwd = password, database  = dbname)
        print("Connection successful!")
    except Error as e:
        print(f"The error {e} has occured.")
    
    return con

def engine(hostname, username, password, dbname, port):
    # Create engine
    eng = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}:{port}/{dbname}')
    return eng

In [4]:
def csv_to_staging(datafile):
    # Create engine using the pre defined engine function
    eng = engine("localhost", "root", "Layaldbroot1997", "feedback_source", "3306")

    # Load csv to dataframe
    df = pd.read_csv(datafile, sep=', ', engine='python')

    # Load dataframe to staging schema
    df.to_sql(name="feedback",con=eng, schema="feedback_source",if_exists="replace",index = False)

csv_to_staging("../Data/sentiment-analysis.csv")

def extract_source():
    # Create connection and cursor
    conn = connection("localhost", "root", "Layaldbroot1997", "feedback_source")
    cursor = conn.cursor()

    # The query extracts all data in the staging schema
    query = "Select * from feedback_source.feedback"
    cursor.execute(query)
    data = cursor.fetchall()

    # Define columns names and load cursor data to a dataframe
    column_names = [i[0] for i in cursor.description]
    df = pd.DataFrame(data, columns=column_names)

    # Extract to a csv file
    df.to_csv("../Data/Extracted.csv", index=False)

extract_source()

Connection successful!


In [5]:
def transform_feedback():
    # Extract the data from staging area
    df = pd.read_csv("../Data/Extracted.csv")

    # Take a copy of the df
    df = df.copy()

    # Drop dupicates
    df =  df.drop_duplicates()

    # Rename columns properly
    df.rename(columns={'"Text': 'Text', 'Confidence Score"': 'Confidence Score'}, inplace=True)

    # Remove characters from values
    df.loc[:,'Text'] = df['Text'].str.replace(r'\"', '', regex=True)
    df.loc[:,'Confidence Score'] = df['Confidence Score'].str.replace(r'\"', '', regex=True)
    df.loc[:,'User ID'] = df['User ID'].str.replace(r'@', '', regex=True)

    # Change some data types
    df['Date/Time'] = pd.to_datetime(df['Date/Time'])
    df['Confidence Score'] = df['Confidence Score'].astype(dtype='float')

    # Save transformed data in a csv
    df.to_csv("../Data/Transformed.csv", index=False)

transform_feedback()

In [6]:
def load_sources():
    # Load transformd data into a dataframe
    df = pd.read_csv("../Data/Transformed.csv")

    # Create engine
    eng = engine("localhost", "root", "Layaldbroot1997", "feedback_dwh", "3306")

    # SQL commands to drop and create foreign key, create priary key and set its type
    drop_fk = text("ALTER TABLE fact_feedback DROP FOREIGN KEY source_id;")
    add_fk = text("""
    ALTER TABLE fact_feedback
    ADD CONSTRAINT source_id
    FOREIGN KEY (source_id) REFERENCES dim_source(id);
    """)
    id_type = text("ALTER TABLE dim_source MODIFY COLUMN id INT;")
    add_pk = text("ALTER TABLE dim_source ADD PRIMARY KEY (id);")

    # Load distinct sources to a dataframe
    sources = df[['Source']].drop_duplicates()

    # Generate ids for the sources
    sources = sources.reset_index(drop=True)
    sources['id'] = sources.index + 1 

    # Organize the columns
    sources = sources[['id', 'Source']]

    with eng.connect() as connection:
        # Drop foreign key
        connection.execute(drop_fk)

        # Load the sources to the corresponding table in the data warehouse
        sources.to_sql(name="dim_source", con=eng, schema="feedback_dwh", if_exists="replace", index=False)

        # Add primary key and set its type
        connection.execute(add_pk)
        connection.execute(id_type)

        # Add foreign key
        connection.execute(add_fk)

    # Save sources into a csv
    sources.to_csv("../Data/sources.csv", index=False)

load_sources()

In [7]:
def load_locations():
    # Load transformd data into a dataframe
    df = pd.read_csv("../Data/Transformed.csv")
    # Create engine
    eng = engine("localhost", "root", "Layaldbroot1997", "feedback_dwh", "3306")

    # SQL commands to drop and create foreign key, create priary key and set its type
    drop_fk = text("ALTER TABLE fact_feedback DROP FOREIGN KEY location_id;")
    add_fk = text("""
    ALTER TABLE fact_feedback
    ADD CONSTRAINT location_id
    FOREIGN KEY (location_id) REFERENCES dim_location(id);
    """)
    id_type = text("ALTER TABLE dim_location MODIFY COLUMN id INT;")
    add_pk = text("ALTER TABLE dim_location ADD PRIMARY KEY (id);")

    # Load distinct locations to a dataframe
    locations = df[['Location']].drop_duplicates()

    # Generate ids for the locations
    locations = locations.reset_index(drop=True)
    locations['id'] = locations.index + 1

    # Organize the columns
    locations = locations[['id', 'Location']]

    with eng.connect() as connection:
        # Drop foreign key
        connection.execute(drop_fk)

        # Load the locations to the corresponding table in the data warehouse
        locations.to_sql(name="dim_location", con=eng, schema="feedback_dwh", if_exists="replace", index=False)

        # Add primary key and set its type
        connection.execute(add_pk)
        connection.execute(id_type)

        # Add foreign key
        connection.execute(add_fk)

    # Save sources into a csv
    locations.to_csv("../Data/locations.csv", index=False)

load_locations()

In [8]:
def load_sentiments():
    # Load transformd data into a dataframe
    df = pd.read_csv("../Data/Transformed.csv")

    # Create engine
    eng = engine("localhost", "root", "Layaldbroot1997", "feedback_dwh", "3306")

    # SQL commands to drop and create foreign key, create priary key and set its type
    drop_fk = text("ALTER TABLE fact_feedback DROP FOREIGN KEY sentiment_id;")
    add_fk = text("""
    ALTER TABLE fact_feedback
    ADD CONSTRAINT sentiment_id
    FOREIGN KEY (sentiment_id) REFERENCES dim_sentiment(id);
    """)
    id_type = text("ALTER TABLE dim_sentiment MODIFY COLUMN id INT;")
    add_pk = text("ALTER TABLE dim_sentiment ADD PRIMARY KEY (id);")

    # Load distinct locations to a dataframe
    sentiments = df[['Sentiment']].drop_duplicates()

    # Generate ids for the locations
    sentiments = sentiments.reset_index(drop=True)
    sentiments['id'] = sentiments.index + 1

    # Organize the columns
    sentiments = sentiments[['id', 'Sentiment']]

    with eng.connect() as connection:
        # Drop foreign key
        connection.execute(drop_fk)

        # Load the locations to the corresponding table in the data warehouse
        sentiments.to_sql(name="dim_sentiment", con=eng, schema="feedback_dwh", if_exists="replace", index=False)

        # Add primary key and set its type
        connection.execute(add_pk)
        connection.execute(id_type)

        # Add foreign key
        connection.execute(add_fk)

    # Save sources into a csv
    sentiments.to_csv("../Data/sentiments.csv", index=False)

load_sentiments()

In [9]:
def load_feedback():
    # Load transformd data into a dataframe
    df = pd.read_csv("../Data/Transformed.csv")

    # Create engine
    eng = engine("localhost", "root", "Layaldbroot1997", "feedback_dwh", "3306")

    # Get dimension tables
    df_locations = pd.read_csv("../Data/locations.csv")
    df_locations.rename(columns={'id':'location_id'},inplace=True)
    df_sources = pd.read_csv("../Data/sources.csv")
    df_sources.rename(columns={'id':'source_id'},inplace=True)
    df_sentiments = pd.read_csv("../Data/sentiments.csv")
    df_sentiments.rename(columns={'id':'sentiment_id'},inplace=True)

    # Merge tables
    merged = pd.merge(df, df_locations,on="Location", how='left')
    merged = pd.merge(merged, df_sentiments, on='Sentiment', how='left')
    merged = pd.merge(merged, df_sources, on='Source', how='left')

    # Generate ids for the feedback
    merged = merged.reset_index(drop=True)
    merged['id'] = merged.index + 1

    # Drop unneeded columns and order columns
    merged.drop(columns=['Sentiment', 'Source', 'Location'], inplace=True)
    merged = merged[['id','Text','sentiment_id', 'Date/Time', 'User ID', 'location_id', 'source_id', 'Confidence Score']]

    # Load the locations to the corresponding table in the data warehouse
    merged.to_sql(name="fact_feedback", con=eng, schema="feedback_dwh", if_exists="replace", index=False)

    # Save feedback into a csv
    merged.to_csv("../Data/feedback.csv", index=False)

load_feedback()